<a href="https://colab.research.google.com/github/blackleg-sanji007/NCRB-Crime-Against-Women-analysis-and-Forecasting/blob/main/NCRB_Crime_Against_Women_analysis_and_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('/content/crime against women.csv')

In [ ]:
print(df.head)

<bound method NDFrame.head of                       state_name  year  total_crimes
0    Andaman And Nicobar Islands  2017            24
1    Andaman And Nicobar Islands  2018            50
2    Andaman And Nicobar Islands  2019            25
3    Andaman And Nicobar Islands  2020             5
4    Andaman And Nicobar Islands  2021            32
..                           ...   ...           ...
211                  West Bengal  2018          4397
212                  West Bengal  2019          4397
213                  West Bengal  2020          3951
214                  West Bengal  2021          3817
215                  West Bengal  2022          3566

[216 rows x 3 columns]>


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Load your DataFrame
# Assuming df is already loaded from a previous cell

actual = []
predicted = []

states = df['state_name'].unique()

for state in states:
    sub_df = df[df['state_name'] == state]

    # We need data for years before 2021 to train and data for 2021 to test.
    # We also need at least two data points for training a linear regression model.
    if not (2021 in sub_df['year'].values and len(sub_df[sub_df['year'] < 2021]) >= 2):
        continue  # Skip if not enough data

    # Train on data before 2021
    train = sub_df[sub_df['year'] < 2021]
    test = sub_df[sub_df['year'] == 2021]

    X_train = train[['year']]
    y_train = train['total_crimes']

    X_test = test[['year']]
    y_test = test['total_crimes']


    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    actual.extend(y_test.tolist())
    predicted.extend(y_pred.tolist())

# Final Evaluation
print("MAE:", mean_absolute_error(actual, predicted))
print("RMSE:", np.sqrt(mean_squared_error(actual, predicted)))
print("R² Score:", r2_score(actual, predicted))

MAE: 323.56944444444446
RMSE: 691.0319627999208
R² Score: 0.9603096338566728


In [ ]:
pip install prophet


In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Load and prepare data
state = 'Rajasthan'
state_df = df[df['state_name'] == state].copy()

state_df['ds'] = pd.to_datetime(state_df['year'].astype(str) + '-12-31')
state_df['y'] = state_df['total_crimes']

# Split train/test
train_df = state_df[state_df['year'] < 2022]
test_df = state_df[state_df['year'] == 2022]

# Train Prophet
model = Prophet()
model.fit(train_df[['ds', 'y']])

# Forecast for 1 year ahead (2021)
future = model.make_future_dataframe(periods=1, freq='YE')
forecast = model.predict(future)

# Get predicted value for 2021
y_pred = forecast[forecast['ds'].dt.year == 2021]['yhat'].values[0]
y_true = test_df['y'].values[0]

# Evaluate
mae = mean_absolute_error([y_true], [y_pred])
rmse = np.sqrt(mean_squared_error([y_true], [y_pred]))
print(f"Actual: {y_true}, Predicted: {round(y_pred)}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpek7qspby/xqqfk8aq.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpek7qspby/v6gikb3h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8414', 'data', 'file=/tmp/tmpek7qspby/xqqfk8aq.json', 'init=/tmp/tmpek7qspby/v6gikb3h.json', 'output', 'file=/tmp/tmpek7qspby/prophet_modelxy23r442/prophet_model-20250704234429.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
23:44:29 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
23:44:29 - cmdstanpy - INFO - Chain [1] done processing
INFO:cm

Actual: 13260, Predicted: 15630
MAE: 2369.665466356808
RMSE: 2369.665466356808


In [1]:
!pip install openai

from openai import OpenAI

In [ ]:
def analysis(data):
          client = OpenAI(
            base_url="https://openrouter.ai/api/v1", #
            api_key="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",
          )
          completion = client.chat.completions.create(
            extra_headers={
              "HTTP-Referer": "<YOUR_SITE_URL>", # Site URL, incase if user wants to integrate this analysis model to a website
              "X-Title": "<YOUR_SITE_NAME>", # Name of site.
            },
            extra_body={},
            model="meta-llama/llama-3.3-70b-instruct:free", #model used for caption analysis
            messages=[
              {
                "role": "user", # tells or defines the identity of enetiy askiing the query. Who is speaking in the conversation.
                "content": f"Analyse the Crime data of the state and try to infer the reasons (social, economic, political, legal factors) behind them. Also analyse the impact of Covid 19 and reasons behind it:\n\n{data}\n\n" # promps (Query) provied to LLM. This is the actual text/message that the 'role' is providing.
              }
            ]
          )
          print(completion.choices[0].message.content)
          return completion.choices[0].message.content #returning the obtained output.